In [31]:
import pandas as pd

# Load the .txt file into a DataFrame
def load_data(filename):
    # Read the file, assuming :::-separated values for the plot and genre
    data = pd.read_csv(filename, sep=':::', header=None, names=['genre', 'plot'])
    return data



# Load the data into a DataFrame
test_data_df = load_data('test_data_solution.txt')
train_data_df = load_data('train_data.txt')




<ipython-input-31-4ce8f2b5531a>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(filename, sep=':::', header=None, names=['genre', 'plot'])
<ipython-input-31-4ce8f2b5531a>:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(filename, sep=':::', header=None, names=['genre', 'plot'])


In [32]:
# Show the first few rows of the DataFrame
test_data_df.head()


,,genre,plot
1,Edgar's Lunch (1998),thriller,"L.R. Brane loves his life - his car, his apar..."
2,La guerra de papá (1977),comedy,"Spain, March 1964: Quico is a very naughty ch..."
3,Off the Beaten Track (2010),documentary,One year in the life of Albin and his family ...
4,Meu Amigo Hindu (2015),drama,"His father has died, he hasn't spoken with hi..."
5,Er nu zhai (1955),drama,Before he was known internationally as a mart...


In [33]:
train_data_df.head()


,,genre,plot
1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his do...
2,Cupid (1997),thriller,A brother and sister with a past incestuous r...
3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fie...
4,The Secret Sin (1915),drama,To help their unemployed father make ends mee...
5,The Unrecovered (2007),drama,The film's title refers not only to the un-re...


In [34]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer

# Function to clean the text data (remove punctuation, lowercase, etc.)
def clean_text(text):
    # Remove punctuation and convert to lowercase
    text = re.sub(r'[^\w\s]', '', text.lower())
    return text

# Apply the clean_text function to the 'Plot' column
train_data_df['cleaned_plot'] = train_data_df['plot'].apply(clean_text)
test_data_df['cleaned_plot'] = test_data_df['plot'].apply(clean_text)

# Show the cleaned text
train_data_df.head()
test_data_df.head()


,,genre,plot,cleaned_plot
1,Edgar's Lunch (1998),thriller,"L.R. Brane loves his life - his car, his apar...",lr brane loves his life his car his apartmen...
2,La guerra de papá (1977),comedy,"Spain, March 1964: Quico is a very naughty ch...",spain march 1964 quico is a very naughty chil...
3,Off the Beaten Track (2010),documentary,One year in the life of Albin and his family ...,one year in the life of albin and his family ...
4,Meu Amigo Hindu (2015),drama,"His father has died, he hasn't spoken with hi...",his father has died he hasnt spoken with his ...
5,Er nu zhai (1955),drama,Before he was known internationally as a mart...,before he was known internationally as a mart...


In [36]:
# Initialize the TF-IDF Vectorizer
tfidf = TfidfVectorizer(max_features=5000, stop_words='english')

# Fit and transform the training data
X_train = tfidf.fit_transform(train_data_df['cleaned_plot'])

# Only transform the test data (don't fit on test data to avoid leakage)
X_test = tfidf.transform(test_data_df['cleaned_plot'])

# Show the shape of the transformed data (X_train and X_test are now feature matrices)
X_train.shape, X_test.shape


((54214, 5000), (54200, 5000))

In [42]:

from sklearn.model_selection import train_test_split

# Assuming `train_data_df` contains both 'plot' (text) and 'genre' (target)
# Split the data into training and validation sets (80% train, 20% validation)
X_train_split, X_val_split, y_train, y_val = train_test_split(
    X_train, train_data_df['genre'], test_size=0.2, random_state=42)

# Check the size of the train and validation splits
X_train_split.shape, X_val_split.shape


((43371, 5000), (10843, 5000))

In [43]:
from sklearn.linear_model import LogisticRegression

# Initialize the Logistic Regression model
model = LogisticRegression(max_iter=1000)  # Increase max_iter if convergence issues arise

# Train the model on the training split
model.fit(X_train_split, y_train)

# Now the model is trained, and you can make predictions on validation data


LogisticRegression(max_iter=1000)

In [44]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Predict on the validation data
y_pred = model.predict(X_val_split)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')

# Print detailed classification report (precision, recall, F1-score)
print(classification_report(y_val, y_pred))

# Optionally, print the confusion matrix
print(confusion_matrix(y_val, y_pred))


Validation Accuracy: 57.61%


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


               precision    recall  f1-score   support

      action        0.54      0.26      0.35       263
       adult        0.75      0.21      0.33       112
   adventure        0.46      0.15      0.23       139
   animation        0.67      0.10      0.17       104
   biography        0.00      0.00      0.00        61
      comedy        0.51      0.58      0.54      1443
       crime        0.33      0.02      0.04       107
 documentary        0.66      0.84      0.74      2659
       drama        0.54      0.77      0.64      2697
      family        0.39      0.07      0.12       150
     fantasy        0.00      0.00      0.00        74
   game-show        0.95      0.45      0.61        40
     history        0.00      0.00      0.00        45
      horror        0.62      0.55      0.59       431
       music        0.65      0.48      0.55       144
     musical        0.50      0.02      0.04        50
     mystery        0.00      0.00      0.00        56
        n

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [45]:
# Predict on the test data
y_test_pred = model.predict(X_test)

# Since the test data may not have labels, you won't be able to calculate accuracy.
# But you can at least output the predictions for inspection.
print(y_test_pred[:10])  # Show the first 10 predictions


[' short ' ' drama ' ' documentary ' ' drama ' ' drama ' ' drama '
 ' drama ' ' comedy ' ' documentary ' ' drama ']


In [52]:
from sklearn.metrics import accuracy_score

# Assuming you have the true test labels
accuracy = accuracy_score(test_data_df['genre'], y_test_pred)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


Test Accuracy: 57.71%
